In [8]:
import ee
import geopandas as gpd
import eeconvert

In [48]:
VECTOR = r'../vector/batagai.shp'
OUTFILE = r'../batagai.csv'
CRS = 'EPSG:32653'

In [47]:
ee.Initialize()

#### Load dataset 

In [68]:
df = gpd.read_file(VECTOR)

#### Convert geodataframe to feature collection 

In [69]:
fc = eeconvert.gdfToFc(df)

#### access GEE Layer

In [70]:
arcticDEM = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic")

In [71]:
output = arcticDEM.select("elevation").sampleRegions(collection=fc)

In [72]:
agg = output.aggregate_array("elevation")

In [73]:
elevation = agg.getInfo()

#### write output 

In [74]:
df['X'] = df.to_crs(crs=CRS).geometry.x
df['Y'] = df.to_crs(crs=CRS).geometry.y
df['Z'] = elevation

In [80]:
df = df.drop(columns=['geometry']).rename(columns={'id':'LABEL'})

In [81]:
df.set_index('LABEL')

,X,Y,Z
LABEL,,,
0,484509.423356,7.504814e+06,125.928513
1,495191.499138,7.495427e+06,176.582077
2,486361.884976,7.494875e+06,194.170074
3,483326.899200,7.500659e+06,138.010605
4,488032.134178,7.503699e+06,211.822952
5,488353.356935,7.496855e+06,460.481506
6,488853.345654,7.494739e+06,329.439423
7,492117.740464,7.491745e+06,285.433655


In [82]:
df.to_csv(OUTFILE, index=False)